<a href="https://colab.research.google.com/github/giyeongyoon/3rd_AGC/blob/master/1_stage_cnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture
!pip install albumentations==1.1.0
!pip install agml

Import libraries

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torch.utils.tensorboard import SummaryWriter
from torchvision import models, transforms
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import os
import cv2
import albumentations as A
import pandas as pd
import matplotlib.pyplot as plt
import time

Download 2021 Autonomous Greenhouse Challenge dataset

In [3]:
import agml
loader = agml.data.AgMLDataLoader('autonomous_greenhouse_regression', dataset_path = './')

[AgML Download]: Extracting files for autonomous_greenhouse_regression... Done!

You have just downloaded autonomous_greenhouse_regression.

This dataset is licensed under the CC BY-SA 4.0 license.
To learn more, visit: https://creativecommons.org/licenses/by-sa/4.0/

When using this dataset, please cite the following:

@misc{https://doi.org/10.4121/15023088.v1,
  doi = {10.4121/15023088.V1},
  url = {https://data.4tu.nl/articles/_/15023088/1},
  author = {Hemming,  S. (Silke) and de Zwart,  H.F. (Feije) and Elings,  A. (Anne) and bijlaard,  monique and Marrewijk,  van,  Bart and Petropoulou,  Anna},
  keywords = {Horticultural Crops,  Mechanical Engineering,  FOS: Mechanical engineering,  Artificial Intelligence and Image Processing,  FOS: Computer and information sciences,  Horticultural Production,  FOS: Agriculture,  forestry and fisheries,  Autonomous Greenhouse Challenge,  autonomous greenhouse,  Artificial Intelligence,  image processing,  computer vision,  Horticulture,  Lettuc

AttributeError: ignored

Define data and output directories

In [4]:
sav_dir='model_weights/'
if not os.path.exists(sav_dir):
    os.mkdir(sav_dir)
# Comment these two lines and uncomment the next two if you've already croppped the images to another directory
RGB_Data_Dir   = './autonomous_greenhouse_regression/images/'
Depth_Data_Dir = './autonomous_greenhouse_regression/depth_images/'


# RGB_Data_Dir='./autonomous_greenhouse_regression/cropped_images/'
# Depth_Data_Dir='./autonomous_greenhouse_regression/cropped_depth_images/'


JSON_Files_Dir = './autonomous_greenhouse_regression/annotations.json'

Crop

In [5]:
# import matplotlib.pyplot as plt
min_x=650
max_x=1450
min_y=200
max_y=900
cropped_img_dir='./autonomous_greenhouse_regression/cropped_images/'

cropped_depth_img_dir='./autonomous_greenhouse_regression/cropped_depth_images/'

if not os.path.exists(cropped_img_dir):
    os.mkdir(cropped_img_dir)

if not os.path.exists(cropped_depth_img_dir):
    os.mkdir(cropped_depth_img_dir)

for im in os.listdir(RGB_Data_Dir):
    img = cv2.imread(RGB_Data_Dir+im)
    crop_img = img[min_y:max_y,min_x:max_x]
    cv2.imwrite(cropped_img_dir+im, crop_img)

for depth_im in os.listdir(Depth_Data_Dir):
    depth_img = cv2.imread(Depth_Data_Dir+depth_im, 0)
    crop_depth_img = depth_img[min_y:max_y,min_x:max_x]
    cv2.imwrite(cropped_depth_img_dir+depth_im, crop_depth_img)

RGB_Data_Dir   = cropped_img_dir
Depth_Data_Dir = cropped_depth_img_dir

Create PyTorch dataset, create PyTorch dataloader, and split train/val/test

In [6]:
split_seed = 12
num_epochs = 400

In [7]:
class GreenhouseDataset(Dataset):
    def __init__(self, rgb_dir, d_dir, jsonfile_dir, rgb_transforms=None, d_transforms=None):

        self.df= pd.read_json(jsonfile_dir)
        # flatten_json is a custom function to flat the nested json files!

        self.rgb_transforms = rgb_transforms
        self.d_transforms = d_transforms
        self.rgb_dir = rgb_dir
        self.d_dir = d_dir
        self.num_outputs = len(self.df.iloc[0]['outputs']['regression'])


    def __getitem__(self, idx):
        # load images
        row=self.df.iloc[idx]

        rgb = plt.imread(self.rgb_dir+row['image'])
        depth = plt.imread(self.d_dir+row['depth_image'])
        depth = np.expand_dims(depth, 2)

        target = list(row['outputs']['regression'].values())

        #make sure your img and mask array are in this format before passing into albumentations transforms, img.shape=[H, W, C]
        if self.rgb_transforms is not None:
            aug_rgb = self.rgb_transforms(image=rgb)
            rgb = aug_rgb['image']
        elif self.d_transforms is not None:
            aug_depth = self.d_transforms(image=depth)
            depth = aug_depth['image']

        rgb = np.transpose(rgb, (2,0,1))
        depth = np.transpose(depth, (2,0,1))

        #pytorch wants a different format for the image ([C, H, W])
        rgb = torch.as_tensor(rgb, dtype=torch.float32)
        depth = torch.as_tensor(depth, dtype=torch.float32)
        target=torch.as_tensor(target, dtype=torch.float32)

        return rgb, depth, target

    def __len__(self):
        return len(self.df)

In [8]:
## FIGURE OUT HOW TO CROP ALL THE IMAGES TO GET RID OF EXTRANIOUS PIXELS
def get_transforms(train, means, stds):
    if train:
        transforms = A.Compose([
        # A.Crop(x_min=650, y_min=200, x_max=1450, y_max=900, always_apply=False, p=1.0),
        A.Flip(p=0.5),
        A.ShiftScaleRotate(always_apply=False, p=0.5, shift_limit=(-0.06, 0.06), scale_limit=(-0.1, 0.1), rotate_limit=(-5, 5), interpolation=0, border_mode=0, value=means, mask_value=None),
        A.Normalize(mean=means, std=stds, max_pixel_value=1.0, always_apply=False, p=1.0)
        ])
    else:
        transforms =  A.Compose([
        # A.Crop(x_min=650, y_min=200, x_max=1450, y_max=900, always_apply=False, p=1.0),
        A.Normalize(mean=means, std=stds, max_pixel_value=1.0, always_apply=False, p=1.0)
        ])
    return transforms

In [9]:
# Instantiate the PyTorch datalaoder the autonomous greenhouse dataset.
dataset = GreenhouseDataset(rgb_dir = RGB_Data_Dir,
                            d_dir = Depth_Data_Dir,
                            jsonfile_dir = JSON_Files_Dir,
                            rgb_transforms = get_transforms(train=False, means=[0,0,0],stds=[1,1,1]),
                            d_transforms = get_transforms(train=False, means=[0,0,0],stds=[1,1,1]))

# Remove last 50 images from training/validation set. These are the test set.
dataset.df= dataset.df.iloc[:-50]

# Split train and validation set. Stratify based on variety.
train_split, val_split = train_test_split(dataset.df,
                                          test_size = 0.2,
                                          random_state = split_seed,
                                          stratify = dataset.df['outputs'].str['classification']) #change to None if you don't have class info
train = torch.utils.data.Subset(dataset, train_split.index.tolist())
val   = torch.utils.data.Subset(dataset, val_split.index.tolist())

# Create train and validation dataloaders
train_loader = torch.utils.data.DataLoader(train, batch_size=6, num_workers=6, shuffle=True)
val_loader   = torch.utils.data.DataLoader(val,   batch_size=6, shuffle=False, num_workers=6)


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 6 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


Determine the mean and standard deviation of images for normalization (Only need to do once for a new dataset)

In [10]:
# this part is just to check the MEAN and STD of the dataset (dont run unless you need mu and sigma)

n_rgb = 0
n_depth = 0
mean_rgb = 0.
std_rgb = 0.
mean_depth = 0.
std_depth = 0.
dataloader = torch.utils.data.DataLoader(dataset, batch_size=5, shuffle=False, num_workers=12)
for rgb, depth, _ in dataloader:

    # Rearrange batch to be the shape of [B, C, W * H]
    rgb = rgb.view(rgb.size(0), rgb.size(1), -1)
    depth = depth.view(depth.size(0), depth.size(1), -1)
    # Update total number of images
    n_rgb += rgb.size(0)
    n_depth += depth.size(0)
    # Compute mean and std here
    mean_rgb += rgb.mean(2).sum(0)
    std_rgb += rgb.std(2).sum(0)
    mean_depth += depth.mean(2).sum(0)
    std_depth += depth.std(2).sum(0)

# Final step
mean_rgb /= n_rgb
std_rgb /= n_rgb
mean_depth /= n_depth
std_depth /= n_depth

print('Mean of RGB: '+ str(mean_rgb))
print('Standard Deviation of RGB', str(std_rgb))
print('Mean of Depth: '+ str(mean_depth))
print('Standard Deviation of Depth', str(std_depth))

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 12 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


Mean of RGB: tensor([0.5482, 0.4620, 0.3602])
Standard Deviation of RGB tensor([0.1639, 0.1761, 0.2659])
Mean of Depth: tensor([0.0127])
Standard Deviation of Depth tensor([0.0035])


Copy the output of the previous cells into here to avoid needing to redetermine mean and std every time

In [11]:
dataset.means = [0.5482, 0.4620, 0.3602, 0.0127]  #these values were copied from the previous cell
dataset.stds = [0.1639, 0.1761, 0.2659, 0.0035]   #copy and paste the values to avoid having
                                                  # to rerun the previous cell for every iteration

Set device

In [12]:
# Set device
device = torch.device('cuda' if torch.cuda.is_available else 'cpu')

Model

In [13]:
class FirstStageModel(nn.Module):
    def __init__(self):
        super(FirstStageModel, self).__init__()
        # RGB Model
        self.rgb_processing_block = nn.Sequential(nn.Conv2d(3, 32, kernel_size=3, stride=2, padding=1),
                                                  nn.Conv2d(32, 3, kernel_size=1),
                                                  nn.AdaptiveAvgPool2d((224, 224)))
        self.rgb_encoder = models.resnet18(pretrained=True)
        self.rgb_regressor = nn.Sequential(nn.Dropout(0.5),
                                           nn.Linear(1000, 256),
                                           nn.ReLU(),
                                           nn.Dropout(0.5),
                                           nn.Linear(256, 3),
                                           nn.ReLU())


        # Depth Model
        self.depth_processing_block = nn.Sequential(nn.Conv2d(1, 32, kernel_size=3, stride=2, padding=1),
                                         nn.Conv2d(32, 1, kernel_size=1),
                                         nn.AdaptiveAvgPool2d((224, 224)))
        self.depth_encoder = models.resnet50(pretrained=False)
        self.depth_encoder.conv1 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)
        self.depth_regressor = nn.Sequential(nn.Dropout(0.5),
                                             nn.Linear(1000, 256),
                                             nn.ReLU(),
                                             nn.Dropout(0.5),
                                             nn.Linear(256, 1),
                                             nn.ReLU())

        self.final = nn.Sequential(nn.Dropout(0.5),
                                   nn.Linear(4, 2048),
                                   nn.ReLU(),
                                   nn.Dropout(0.5),
                                   nn.Linear(2048, 2048),
                                   nn.ReLU(),
                                   nn.Dropout(0.5),
                                   nn.Linear(2048, 3),
                                   nn.ReLU())

    def forward(self, rgb, depth):
        rgb_out = self.rgb_processing_block(rgb)
        rgb_out = self.rgb_encoder(rgb_out)
        rgb_out = self.rgb_regressor(rgb_out)

        depth_out = self.depth_processing_block(depth)
        depth_out = self.depth_encoder(depth_out)
        output2 = self.depth_regressor(depth_out)  # height

        output1 = torch.cat([rgb_out, output2], dim=1)
        output1 = self.final(output1)  # fresh weight, dry weight, diameter

        return output1, output2

In [14]:
model = FirstStageModel()

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:01<00:00, 57.6MB/s]
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed 

Hyperparameter

In [15]:
lr = 0.0005
epochs = 200
batch_size = 32

NMSE loss

In [16]:
class NMSELoss(nn.Module):
    def __init__(self):
          # super(diceloss, self).init()
        super(NMSELoss, self).__init__()
          # print('HI')
    def forward(self, pred, target):
        if target.size() != pred.size():
              raise ValueError("Target size ({}) must be the same as input size ({})".format(target.size(), pred.size()))

        num=torch.sum((target-pred)**2,0)
        den=torch.sum(target**2,0)

        return torch.sum(num/den)

Loss and optimizer

In [30]:
criterion = NMSELoss()
# optimizer = optim.Adam(model.parameters(), lr=lr)
optimizer = torch.optim.Adam(model.parameters(),
                            lr=lr,
                            betas=(0.9, 0.999),
                            eps=1e-08,
                            weight_decay = 0,
                            amsgrad = False)

Train

In [31]:
def train_single_epoch(model, dataset, device,
                       criterion, optimizer,
                       writer, epoch, train_loader):
    model.train()

    dataset.rgb_transforms = get_transforms(train=True, means=dataset.means[:3], stds=dataset.stds[:3])
    dataset.d_transforms = get_transforms(train=True, means=dataset.means[3:], stds=dataset.stds[3:])

    for i, (rgb, depth, label) in enumerate(train_loader):
        rgb = rgb.to(device)
        depth = depth.to(device)
        label = label.to(device)  # ['FreshWeightShoot', 'DryWeightShoot', 'Height', 'Diameter', 'LeafArea']

        # Forward pass - First stage
        pred1, pred2 = model(rgb, depth)  # pred1: fresh weight, dry weight, diameter
                                                      # pred2: height

        # Calculate loss
        pred = torch.cat([pred1[:, :2], pred2], dim=1)
        pred = torch.cat([pred, pred1[:, 2:]], dim=1)
        loss = criterion(pred, label[:, :4])

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        print(f'Epoch {epoch+1}/{epochs}, Batch {i+1}/{len(train_loader)}, Loss: {loss.item()}')
        with open('run.txt', 'a') as f:
            f.write('\n')
            f.write('Train MSE: '+ str(loss.tolist()))


In [32]:
def validate(model, dataset, device, sav_dir, criterion, writer, epoch, val_loader, best_val_loss):
    current_val_loss = 0
    # training_val_loss=0s

    model.eval()
    print('Validating and Checkpointing!')

    dataset.rgb_transforms = get_transforms(train=True, means=dataset.means[:3], stds=dataset.stds[:3])
    dataset.d_transforms = get_transforms(train=True, means=dataset.means[3:], stds=dataset.stds[3:])

    with torch.no_grad():
        for i, (rgb, depth, label) in enumerate(val_loader):
            rgb = rgb.to(device)
            depth = depth.to(device)
            label = label.to(device)

            pred1, pred2 = model(rgb, depth)

            pred = torch.cat([pred1[:, :2], pred2], dim=1)
            pred = torch.cat([pred, pred1[:, 2:]], dim=1)
            loss = criterion(pred, label[:, :4])
            # acc=nmse(preds.detach(), targets)
            current_val_loss = current_val_loss + loss.item()
            # training_val_loss=training_val_loss+loss.detach().cpu().numpy()

        # writer.add_scalar("MSE Loss/val", training_val_loss, epoch)
        writer.add_scalar("MSE Loss/val", current_val_loss, epoch)

    if current_val_loss < best_val_loss or epoch == 0:
        best_val_loss = current_val_loss
        torch.save(model.state_dict(), sav_dir+'bestmodel' + '.pth')
        print('Best model Saved! Val MSE: ', str(best_val_loss))
        with open('run.txt', 'a') as f:
            f.write('\n')
            f.write('Best model Saved! Val MSE: '+ str(best_val_loss))

    else:
        print('Model is not good (might be overfitting)! Current val MSE: ', str(current_val_loss), 'Best Val MSE: ', str(best_val_loss))
        with open('run.txt', 'a') as f:
            f.write('\n')
            f.write('Model is not good (might be overfitting)! Current val MSE: '+ str(current_val_loss)+ 'Best Val MSE: '+ str(best_val_loss))
    return best_val_loss

In [33]:
model.to(device)

best_val_loss = 9999999 # initial dummy value
current_val_loss = 0

writer = SummaryWriter()
start = time.time()

for epoch in range(epochs):
    with open('run.txt', 'a') as f:
                f.write('\n')
                f.write('Epoch: '+ str(epoch + 1) + ', Time Elapsed: '+ str((time.time()-start)/60) + ' mins')
    print('Epoch: ', str(epoch + 1), ', Time Elapsed: ', str((time.time()-start)/60), ' mins')
    train_single_epoch(model, dataset, device,
                        criterion, optimizer,
                        writer, epoch, train_loader)
    best_val_loss = validate(model, dataset, device, sav_dir,
                                criterion, writer, epoch, val_loader, best_val_loss)

Epoch:  1 , Time Elapsed:  6.2068303426106775e-06  mins


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 6 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
Epoch 96/200, Batch 40/45, Loss: 1.7116940021514893
Epoch 96/200, Batch 41/45, Loss: 2.3889505863189697
Epoch 96/200, Batch 42/45, Loss: 3.0428314208984375
Epoch 96/200, Batch 43/45, Loss: 2.5612220764160156
Epoch 96/200, Batch 44/45, Loss: 1.2447409629821777
Epoch 96/200, Batch 45/45, Loss: 34.119869232177734
Validating and Checkpointing!
Model is not good (might be overfitting)! Current val MSE:  28.008179664611816 Best Val MSE:  25.10259437561035
Epoch:  97 , Time Elapsed:  31.394042102495828  mins
Epoch 97/200, Batch 1/45, Loss: 2.6198554039001465
Epoch 97/200, Batch 2/45, Loss: 3.1290123462677
Epoch 97/200, Batch 3/45, Loss: 3.134857177734375
Epoch 97/200, Batch 4/45, Loss: 3.4151227474212646
Epoch 97/200, Batch 5/45, Loss: 2.3956820964813232
Epoch 97/200, Batch 6/45, Loss: 2.9125547409057617
Epoch 97/200, Batch 7/45, Loss: 3.142975330352783
Epoch 97/200, Batch 8/45, Loss: 3.1159141063690186
Epoch 97/200, Batch 9/45, Loss: 2.716146230697632
Epoc